In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
from pprint import pprint
import time
import io
import os
import re
from pprint import pprint
g = 0

firstyear = 1915
finalyear = 1915
league = 'F'

just_this_team = ''# ['BLF', 'BRF', 'BUF', 'CHF', 'KCF', 'NEW', 'PTF', 'SLF']

pandas.set_option('display.max_colwidth', None)

eventdir = '../input/baseball-events-from-retrosheetorg/'

processed_data_dir = '../input/baseball-events-from-retrosheetorg/processed/'

save_fielders = True

print('initializing dataframes...')
event_id_columns = ['eventID']
gamecolumns = ['gameID']
info_columns = gamecolumns + ['visteam', 'hometeam', 'site', 'date', 'number', 'starttime', 'daynight', 'usedh', 'innings', 'tiebreaker', 'umphome', 'ump1b', 'ump2b', 'ump3b', 'umplf', 'umprf', 'inputtime', 'howscored', 'pitches', 'oscorer', 'temp', 'winddir', 'windspeed', 'fieldcond', 'precip', 'sky', 'timeofgame', 'attendance', 'wp', 'lp', 'save', 'comments']
player_columns = ['playerID', 'visitor_or_home', 'batting_order', 'fielding_position']

event_columns = event_id_columns + gamecolumns
event_columns += ['event_in_game', 'inning', 'visitor_or_home', 'batterID', 'batter_hand']
event_columns += ['pitch_count', 'pitch_sequence', 'theplay']
event_columns += ['pitcherID', 'pitcher_hand', 'badj', 'padj', 'ladj']
if (save_fielders):
    event_columns += ['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID', 'field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID', 'bat_DH_playerID']
event_columns += ['pinch_runner_playerID', 'pinch_runner_replaced_playerID']#, 'badj', 'padj']
event_columns += ['comments']

def ordinaltg(n):
    return str(n) + {1: 'st', 2: 'nd', 3: 'rd'}.get(4 if 10 <= n % 100 < 20 else n % 10, "th")

g = 0
print('Packages loaded!')

initializing dataframes...
Packages loaded!


In [2]:
def fix_special_case_hands(events_df):
    events_df.loc[events_df['batterID'] == 'bradh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'browm102', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'conls101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coopc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'crand101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eaklc101', 'batter_hand'] = '?'
    events_df.loc[events_df['batterID'] == 'englc103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hartg102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mannl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'simom102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wistt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'blaiw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chadc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'conls101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'esmoj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flacm101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gingj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kaufb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'kerrd102', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'knete101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lordh101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'maina101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mckeb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'quinj102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'browm102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'crand101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'conls101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'flacm101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gingj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'knete101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'maina101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'quinj102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gagne101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'whitc102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'woodd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gagne101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitc102', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'woodd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rheac101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'delaf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'reedt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'delaf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'mannl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bradh101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'simom102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'blaiw101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'mckeb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'rheac101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hartg102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'millh102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'harso101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'neffd101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'parkp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walsd101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'harso101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'parkp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'walsd101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brieb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bushd102', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'cartp101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cotte101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dalyt102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'daush101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gregv101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hamie101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'jackj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'koobe101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lappj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lewid101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lewid101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mortg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mullc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'russr102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'speat101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'younr103', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'cartp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cotte101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'dalyt102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'daush101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'engej101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'gregv101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'hamie101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'koobe101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'mortg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'russr102', 'pitcher_hand'] = 'L'    
    events_df.loc[events_df['batterID'] == 'bresr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'engej101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lowdg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bresr101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'lappj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'lowdg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'mullc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'younr103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eschj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'eschj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bradn101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bradn101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hendt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'brieb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'comsr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crans102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'richj103', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'comsr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'richj103', 'pitcher_hand'] = 'R'    
    events_df.loc[events_df['batterID'] == 'ecclh101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'ecclh101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'barbt101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'jamic101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'beckb101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'braif101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chenl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'demaa101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dougp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'egand102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fleta101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hargb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jameb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'johnd107', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lobeh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mccal102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'robed103', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'schuj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'snydf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'beckb101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'braif101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'chenl101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'demaa101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dougp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'jameb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'mccal102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'schuj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dolac102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mclap101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'shanj102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stanp101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wagnj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dolac102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'mclap101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'stanp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'collz101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'stanp101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'leark101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'shanj102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'oescj101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hargb101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'oescj101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corrr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'corrr101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'leark101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'callr101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'callr101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'wagnj101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'robed103', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'schef104', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'egand102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clarf101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'daubh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'boarc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lamlf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'lamlf101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'boarc101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'schef104', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'englc103', 'pitcher_hand'] = 'R'
    
    #1916
    events_df.loc[events_df['batterID'] == 'chapr101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'culln101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fostr103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fullf101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'gallb103', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'henro101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'highh101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'judgj101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'kingl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lavad101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'maysc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'nunal101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'seveh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'shanh101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walkt101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weavb101', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'culln101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'danfd101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'fostr103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gallb103', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'maysc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'boonl101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'danfd101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'grimc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hasbz101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'morig101', 'batter_hand'] = '?'
    events_df.loc[events_df['batterID'] == 'morig101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'groob101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'berga101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'klepe101', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'klepe101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'daltj101', 'batter_hand'] = 'R'    
    events_df.loc[events_df['batterID'] == 'groob101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'morrb102', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'morrb102', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clemv101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bendc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bohns101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coopw101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cravg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cutsg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dooir101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'everj102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fiscw101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'getzg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'herzb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'holkw101', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'jasph101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'killb101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'knabo101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'marar101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'merkf101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ruckn101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'schua102', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tylel101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'warmc101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'warnh102', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zimmh101', 'batter_hand'] = 'R'    
    events_df.loc[events_df['pitcherID'] == 'bendc101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coopw101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'jasph101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ruckn101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'schua102', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'tylel101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'warmc101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'currm101', 'batter_hand'] = 'R'    
    events_df.loc[events_df['pitcherID'] == 'currm101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chapl101', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'clemc101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gleab101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'daleg101', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'pearg101', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'daleg101', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'pearg101', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'batsb101', 'batter_hand'] = 'L'
    
    return events_df
print('ok')

ok


In [3]:
s = time.time()
game_info_all_df = pandas.DataFrame(data=None, columns=info_columns).set_index('gameID')
game_data_all_df = pandas.DataFrame(data=None, columns=['gameID'])#.set_index('gameID')
er_all_df = pandas.DataFrame(data=None, columns=['gameID', 'pitcher', 'er'])

events_all_df = pandas.DataFrame(data=None, columns=event_columns).set_index('eventID')

players_all_df = pandas.DataFrame(data=None, columns=player_columns).set_index(['playerID'])
watchflag = False

comments_at_events = []
pinch_runners_at_events = []
badj_at_events = []
padj_at_events = []
ladj_at_events = []

all_lines_list = []

for i in range(firstyear, finalyear+1):
    
    game_info_thisyear_df = pandas.DataFrame(data=None, columns=game_info_all_df.columns)
#    game_data_thisyear_df = pandas.DataFrame(data=None, columns=game_data_all_df.columns)
    er_thisyear_df = pandas.DataFrame(data=None, columns=er_all_df.columns)
    players_thisyear_df = pandas.DataFrame(data=None, columns=players_all_df.columns)
    events_thisyear_df = pandas.DataFrame(data=None, columns=events_all_df.columns)
    
    thisyear_files = sorted(['{0:}{1:.0f}seve/'.format(eventdir, np.floor(i/10)*10)+x for x in os.listdir('{0:}{1:.0f}seve/'.format(eventdir, np.floor(i/10)*10)) if ((str(i) in x) & (x[-4:] == '.EV{0:}'.format(league)))])
    print('Found {0:,.0f} event files...'.format(len(thisyear_files)))
    for thisfile in thisyear_files:
        game_info_df = pandas.DataFrame(data=None, columns=game_info_thisyear_df.columns)
#        game_data_df = pandas.DataFrame(data=None, columns=game_data_thisyear_df.columns)
        er_df = pandas.DataFrame(data=None, columns=er_thisyear_df.columns)
        events_df = pandas.DataFrame(data=None, columns=events_thisyear_df.columns)
        players_df = pandas.DataFrame(data=None, columns=players_thisyear_df.columns)

        all_lines_list = []
        print('Reading {0:}...'.format(thisfile.split('/')[-1]))
#     if (np.mod(i, 100) == 0):
#         print('\tReading file {0:,.0f} of {1:,.0f}...'.format(i+1, len(eventfiles)))
        with io.open(thisfile, 'r', encoding='utf-8') as f:
            orig_lines_list = f.readlines()
        for this_orig_line in orig_lines_list:
            all_lines_list += [x for x in this_orig_line.split('\n')]
        all_lines_list = [x for x in all_lines_list 
                          if ((x != '') and (x != '\x1a'))
                         ]

        #print('\tRead {0:,.0f} lines...'.format(len(all_lines_list)))
        
        event_dict = {}
        gamecount = 0
        eventcount = 0
        the_event_in_game = 0
        batting_team = 0
        comment_text = ''
        game_info_comment_text = ''

        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
        staged_play = False
        
        for i in range(0,len(all_lines_list)):
        
            if (np.mod(i,1000) == 0):
                print('\tParsing line {0:,.0f} of {1:,.0f}...'.format(i, len(all_lines_list)))
            record_type = all_lines_list[i].split(',')[0]
            if (record_type == 'id'):
                if (all_lines_list[i].split(',')[1] == 'BLF191506250'):
                    #print('\n')
                    watchflag = False
                if (all_lines_list[i].split(',')[1] == 'BLF191506261'):
                    watchflag = False
                if (gamecount >= 1):   #WRITE LAST PLAY OF PREVIOUS GAME!
                    if (comment_text != ''):
                        event_dict['comments'] = comment_text
                    else:
                        event_dict['comments'] = np.nan
                    event_dict['pinch_runner_playerID'] = pinch_runner
                    event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
                    if (pinch_runner_flag == True):
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
                    events_df.loc[eventcount] = event_dict
                    eventcount += 1
                    the_event_in_game == 1
                    comment_text = ''

                    if (game_info_comment_text != ''):
                        this_game_info['comments'] = game_info_comment_text
                        game_info_comment_text = ''
                    else:
                        this_game_info['comments'] = np.nan
                    game_info_df.loc[thegameID] = this_game_info

                gamecount += 1        
                thegameID = all_lines_list[i].split(',')[1]    


                players_df = players_df[players_df['visitor_or_home'].apply(lambda x: True == False)]

        #         if (thegameID == 'ANA202008100'):
        #             break
                eventcount += 1
                this_game_info = {}
                the_event_in_game = 1
                staged_play = False
                #if (np.mod(gamecount, 50) == 1):
                if (np.mod(gamecount, 20) == 1):
                    print('\t\tParsing game {0:.0f}: {1:} (i = {2:,.0f}; eventcount = {3:,.0f})...'.format(gamecount, thegameID, i, eventcount))
        #            print('Starting Top of the 1st...')
            elif (record_type == 'version'):
                this_game_info['version'] = all_lines_list[i].split(',')[1]
            elif (record_type == 'info'):
                this_game_info[all_lines_list[i].split(',')[1]] = all_lines_list[i].split(',')[2]
            elif (record_type == 'start'):
                this_player_df = pandas.DataFrame(data=[[all_lines_list[i].split(',')[1], int(all_lines_list[i].split(',')[3]), int(all_lines_list[i].split(',')[4]), int(all_lines_list[i].split(',')[5])]], columns=player_columns).set_index('playerID')
                players_df = pandas.concat((players_df, this_player_df), axis=0)
            elif (record_type == 'data'):
                this_er_row_df = pandas.DataFrame(data=[[thegameID, all_lines_list[i].split(',')[2], int(all_lines_list[i].split(',')[3])]], columns=er_df.columns)
                er_df = pandas.concat((er_df, this_er_row_df))
            elif (record_type == 'play'):
                if ((watchflag) & (all_lines_list[i].split(',')[-1] == 'NP')):
                    print('{0:0d}: {1:}: {2:}'.format(i, all_lines_list[i], all_lines_list[i].split(',')[-1]))
                if (staged_play):
                    if (comment_text != ''):
                        event_dict['comments'] = comment_text
                    else:
                        event_dict['comments'] = np.nan
                    event_dict['pinch_runner_playerID'] = pinch_runner
                    event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
                    if (pinch_runner_flag == True):
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
                    events_df.loc[eventcount] = event_dict
                    eventcount += 1
                    the_event_in_game == 1
                    comment_text = ''
                event_dict['event_in_game'] = the_event_in_game
                event_dict['gameID'] = thegameID
                event_dict['inning'] = int(all_lines_list[i].split(',')[1])
                event_dict['visitor_or_home'] = int(all_lines_list[i].split(',')[2])
                event_dict['batterID'] = all_lines_list[i].split(',')[3]
                event_dict['pitch_count'] = all_lines_list[i].split(',')[4]
                event_dict['pitch_sequence'] = all_lines_list[i].split(',')[5]
                event_dict['theplay'] = all_lines_list[i].split(',')[6]        
                event_dict['pitcherID'] = players_df[
                    (players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) 
                    & (players_df['fielding_position'] == 1)
                ].index.values[0]
                #print('\t{0:}'.format(event_dict['pitcherID']))
                if (save_fielders):
                    event_dict['field_C_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 2)].index.values[0]
                    event_dict['field_1B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 3)].index.values[0]
                    event_dict['field_2B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 4)].index.values[0]
                    event_dict['field_3B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 5)].index.values[0]
                    event_dict['field_SS_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 6)].index.values[0]
                    event_dict['field_LF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 7)].index.values[0]
                    event_dict['field_CF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 8)].index.values[0]
                    event_dict['field_RF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 9)].index.values[0]
                    #print(this_game_info['usedh'])
                    if (this_game_info['usedh'] == 'true'):  # lol it's a string, not a boolean
                        event_dict['bat_DH_playerID'] = players_df[(players_df['visitor_or_home'] == event_dict['visitor_or_home']) & (players_df['fielding_position'] == 10)].index.values[0]
                if (batting_team != event_dict['visitor_or_home']):
        #             if (event_dict['visitor_or_home'] == 0):
        #                 inningside = 'Top'
        #             else:
        #                 inningside = 'Bottom'
        #             print('Starting {0:} of the {1:} - {2:} pitching...'.format(inningside, ordinal(event_dict['inning']), event_dict['pitcherID']))
                    batting_team = 1 - batting_team
                staged_play = True
                the_event_in_game += 1
            elif (record_type == 'com'):
                if (the_event_in_game <= 1):
                    game_info_comment_text += all_lines_list[i][4:]
                if (eventcount not in comments_at_events):
                    comments_at_events.append(eventcount)
                if (staged_play):
                    if (comment_text != ''):
                        comment_text += ' '
                    comment_text += all_lines_list[i][4:]#.replace('"', '').replace('$', '').replace('.', '')
            elif (record_type == 'sub'):
                if (watchflag):
                    print('\t{0:0d}: {1:}'.format(i, all_lines_list[i]))
                #print(players_df)
                #print('\t\t{0:}'.format(all_lines_list[i]))
                players_df = players_df.reset_index()
                sub_playerID = all_lines_list[i].split(',')[1]
                sub_team = int(all_lines_list[i].split(',')[3])
                sub_batting_order = int(all_lines_list[i].split(',')[4])
                sub_fielding_position = int(all_lines_list[i].split(',')[5])
                if (sub_team == batting_team):
                    if (sub_fielding_position == 11):
                        pass
                        #event_dict['batterID'] = sub_playerID
                    elif (sub_fielding_position == 12):
                        pinch_runners_at_events.append(eventcount)
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [True, sub_playerID, players_df[(players_df['visitor_or_home'] == batting_team) & (players_df['batting_order'] == sub_batting_order)]['playerID'].values[0]]
                    else:
                        players_df.loc[(players_df['visitor_or_home'] == batting_team) & (players_df['batting_order'] == sub_batting_order), 'playerID'] = sub_playerID
                else:
                    players_df.loc[
                        (players_df['visitor_or_home'] == 1-batting_team) 
                        & (players_df['fielding_position'] == sub_fielding_position), 'playerID'] = sub_playerID
                players_df = players_df.set_index('playerID')
            elif (record_type == 'radj'):
                pinch_runners_at_events.append(eventcount)
        #         print('-------------------- RADJ ---------------------------')
        #         print(all_lines_list[i])
                [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [True, all_lines_list[i].split(',')[1], 'radj']
        #         print(pinch_runner_flag, pinch_runner, pinch_runner_replaced)
        #         print('-------------------- /RADJ ---------------------------')
            elif (record_type == 'badj'):

                if (the_event_in_game == 1):
                    #print('\tbadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    badj_at_events.append(eventcount)
                else:
                    #print('\tbadj at eventcount {0:,.0f}: {1:}'.format(eventcount+1, all_lines_list[i]))
                    badj_at_events.append(eventcount+1)
            elif (record_type == 'padj'):
                if (the_event_in_game == 1):
                    #print('\tpadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    padj_at_events.append(eventcount)
                else:
                    #print('\tpadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    padj_at_events.append(eventcount+1)
            elif (record_type == 'ladj'):
                #print('\tladj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                #print(all_lines_list[i])
                ladj_at_events.append(eventcount)
            else:
                print('--------------')
                print(all_lines_list[i-1])     
                print(all_lines_list[i])     
                print(all_lines_list[i+1])     
                print('--------------')
                print('--------------')

        # record last play of final game
        if (comment_text != ''):
            event_dict['comments'] = comment_text
        else:
            event_dict['comments'] = np.nan
        event_dict['pinch_runner_playerID'] = pinch_runner
        event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
        if (pinch_runner_flag == True):
            [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
        events_df.loc[eventcount] = event_dict
        eventcount += 1
        the_event_in_game == 1
        comment_text = ''        

        # record final game info and data
        if (game_info_comment_text != ''):
            this_game_info['comments'] = game_info_comment_text
        else:
            this_game_info['comments'] = np.nan
        game_info_df.loc[thegameID] = this_game_info
        
        events_df.loc[badj_at_events, 'badj'] = True        
        events_df.loc[padj_at_events, 'padj'] = True
        
        game_info_thisyear_df = pandas.concat((game_info_thisyear_df, game_info_df), axis=0)
#        game_data_thisyear_df = pandas.concat((game_data_thisyear_df, game_data_df), axis=0)
        er_thisyear_df = pandas.concat((er_thisyear_df, er_df), axis=0)
        players_thisyear_df = pandas.concat((players_thisyear_df, players_df), axis=0)
        
        events_thisyear_df = pandas.concat((events_thisyear_df, events_df), axis=0)

        print('\n')
        
    game_info_all_df = pandas.concat((game_info_all_df, game_info_thisyear_df), axis=0)
#    game_data_all_df = pandas.concat((game_data_all_df, game_data_thisyear_df), axis=0)
    er_all_df = pandas.concat((er_all_df, er_thisyear_df), axis=0)
    players_all_df = pandas.concat((players_all_df, players_thisyear_df), axis=0)

    events_all_df = pandas.concat((events_all_df, events_thisyear_df), axis=0)

    
print('Killing in-progress events and replacing with the full file...')

game_info_df = game_info_df[game_info_df['number'].apply(lambda x: True == False)]
er_df = er_df[er_df['er'].apply(lambda x: True == False)]
players_df = players_df[players_df['visitor_or_home'].apply(lambda x: True == False)]
events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]

game_info_df = game_info_all_df
er_df = er_all_df
players_df = players_all_df
events_df = events_all_df

er_df = er_df.reset_index(drop=True)
events_df = events_df.reset_index(drop=True)

game_info_df.index.name = 'gameID'
er_df.index.name = 'erID'
players_df.index.name = 'playerID'
events_df.index.name = 'eventID'

# print('backing up...')
game_info_df_bk = game_info_df
er_df_bk = er_df
players_df_bk = players_df
events_df_bk = events_df

e = time.time()
g = g + (e-s)
print('\n')
print('Parsed {0:,.0f} events in {1:,.0f} minutes {2:,.0f} seconds!'.format(len(events_df), np.floor((e-s)/60), (e-s)%60))


Found 8 event files...
Reading 1915BLF.EVF...
	Parsing line 0 of 9,064...
		Parsing game 1: BLF191504100 (i = 0; eventcount = 1)...
	Parsing line 1,000 of 9,064...
	Parsing line 2,000 of 9,064...
		Parsing game 21: BLF191506102 (i = 2,990; eventcount = 1,783)...
	Parsing line 3,000 of 9,064...
	Parsing line 4,000 of 9,064...
	Parsing line 5,000 of 9,064...
		Parsing game 41: BLF191507060 (i = 5,964; eventcount = 3,535)...
	Parsing line 6,000 of 9,064...
	Parsing line 7,000 of 9,064...
	Parsing line 8,000 of 9,064...
		Parsing game 61: BLF191510021 (i = 8,777; eventcount = 5,192)...
	Parsing line 9,000 of 9,064...


Reading 1915BRF.EVF...
	Parsing line 0 of 8,430...
		Parsing game 1: BRF191504100 (i = 0; eventcount = 1)...
	Parsing line 1,000 of 8,430...
	Parsing line 2,000 of 8,430...
	Parsing line 3,000 of 8,430...
		Parsing game 21: BRF191506080 (i = 3,055; eventcount = 1,818)...
	Parsing line 4,000 of 8,430...
	Parsing line 5,000 of 8,430...
		Parsing game 41: BRF191508090 (i = 5,93

In [4]:
s = time.time()

# print('getting from backingup...')
game_info_df = game_info_df[game_info_df['number'].apply(lambda x: True == False)]
er_df = er_df[er_df['er'].apply(lambda x: True == False)]
players_df = players_df[players_df['visitor_or_home'].apply(lambda x: True == False)]
events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]
game_info_df = game_info_df_bk
er_df = er_df_bk
players_df = players_df_bk
events_df = events_df_bk


# events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]
# events_df = events_df_bk

#print('getting roster file...')
rosters_df = pandas.read_csv(processed_data_dir+'rosters.csv'.format(firstyear, finalyear), index_col='rownumber')
events_df = events_df.assign(theyear = pandas.to_numeric(events_df['gameID'].apply(lambda x: x[3:7]), errors='coerce', downcast='integer'))

# events_df = events_df.reset_index().assign(visteam = events_df.reset_index()[['eventID','gameID']].merge(game_info_df.reset_index(drop=True)[['gameID', 'visteam']]).set_index('eventID')['visteam'])
# events_df = events_df.reset_index().assign(hometeam = events_df.reset_index()[['eventID','gameID']].merge(game_info_df.reset_index(drop=True)[['gameID', 'hometeam']]).set_index('eventID')['hometeam'])

# game_info_df = game_info_all_df
# er_all_df = er_all_df.reset_index(drop=True)
# #er_all_df.index.name = 'erID'
# er_all_df
events_df = events_df.assign(visteam = events_df.reset_index().merge(game_info_df.reset_index()[['gameID', 'visteam']], how='left', on='gameID').set_index('eventID')['visteam'])
events_df = events_df.assign(hometeam = events_df.reset_index().merge(game_info_df.reset_index()[['gameID', 'hometeam']], how='left', on='gameID').set_index('eventID')['hometeam'])

#print('finding batter handedness...')
events_df = events_df.assign(batter_hand = np.nan)
events_df.loc[(events_df['visitor_or_home'] == 0), 'batter_hand'] = events_df[(events_df['visitor_or_home'] == 0)].reset_index()[['eventID', 'batterID', 'theyear', 'visteam']].merge(rosters_df.reset_index()[['playerID', 'theyear', 'team', 'batting_hand']], how='left', left_on=['batterID', 'theyear', 'visteam'], right_on=['playerID', 'theyear', 'team']).set_index('eventID')['batting_hand']
events_df.loc[(events_df['visitor_or_home'] == 1), 'batter_hand'] = events_df[(events_df['visitor_or_home'] == 1)].reset_index()[['eventID', 'batterID', 'theyear', 'hometeam']].merge(rosters_df.reset_index()[['playerID', 'theyear', 'team', 'batting_hand']], how='left', left_on=['batterID', 'theyear', 'hometeam'], right_on=['playerID', 'theyear', 'team']).set_index('eventID')['batting_hand']

#print('finding pitcher handedness...')
events_df = events_df.assign(pitcher_hand = np.nan)
events_df.loc[(events_df['visitor_or_home'] == 1), 'pitcher_hand'] = events_df[(events_df['visitor_or_home'] == 1)].reset_index()[['eventID', 'pitcherID', 'theyear', 'visteam']].merge(rosters_df.reset_index()[['playerID', 'theyear', 'team', 'throwing_hand']], how='left', left_on=['pitcherID', 'theyear', 'visteam'], right_on=['playerID', 'theyear', 'team']).set_index('eventID')['throwing_hand']
events_df.loc[(events_df['visitor_or_home'] == 0), 'pitcher_hand'] = events_df[(events_df['visitor_or_home'] == 0)].reset_index()[['eventID', 'pitcherID', 'theyear', 'hometeam']].merge(rosters_df.reset_index()[['playerID', 'theyear', 'team', 'throwing_hand']], how='left', left_on=['pitcherID', 'theyear', 'hometeam'], right_on=['playerID', 'theyear', 'team']).set_index('eventID')['throwing_hand']

#print('fixing identified special cases...')
events_df = fix_special_case_hands(events_df)

#print('accounting for switch hitters...')
events_df.loc[(events_df['batter_hand'] == 'B') & (events_df['pitcher_hand'] == 'R'), 'batter_hand'] = 'L'
events_df.loc[(events_df['batter_hand'] == 'B') & (events_df['pitcher_hand'] == 'L'), 'batter_hand'] = 'R'

#print('applying adjustments...')
#print('\n')
pitcher_hand_switcher_df = pandas.DataFrame(data=events_df[(events_df['padj'] == True)]['pitcher_hand'], columns=['pitcher_hand'])
pitcher_hand_switcher_df = pitcher_hand_switcher_df.assign(pitcher_hand_switched = np.nan)
pitcher_hand_switcher_df.loc[pitcher_hand_switcher_df['pitcher_hand'] == 'L', 'pitcher_hand_switched'] = 'R'
pitcher_hand_switcher_df.loc[pitcher_hand_switcher_df['pitcher_hand'] == 'R', 'pitcher_hand_switched'] = 'L'
events_df.loc[(events_df['padj'] == True), 'pitcher_hand'] = pitcher_hand_switcher_df['pitcher_hand_switched']

batter_hand_switcher_df = pandas.DataFrame(data=events_df[(events_df['badj'] == True)]['batter_hand'], columns=['batter_hand'])
batter_hand_switcher_df = batter_hand_switcher_df.assign(batter_hand_switched = np.nan)
batter_hand_switcher_df.loc[batter_hand_switcher_df['batter_hand'] == 'L', 'batter_hand_switched'] = 'R'
batter_hand_switcher_df.loc[batter_hand_switcher_df['batter_hand'] == 'R', 'batter_hand_switched'] = 'L'
events_df.loc[(events_df['badj'] == True), 'batter_hand'] = batter_hand_switcher_df['batter_hand_switched']



print('No batter hand found for {0:,.0f} batters...'.format(len(events_df[events_df['batter_hand'].isnull()]['batterID'].drop_duplicates())))
# print('\n')
# print(events_df.groupby('batter_hand').size())

missing_hand_batters_list = events_df[events_df['batter_hand'].isnull()]['batterID'].drop_duplicates().sort_values().tolist()
missing_hand_batters_list

for ix, thisrow in rosters_df[
    (rosters_df['playerID'].isin(missing_hand_batters_list)) 
    ][['playerID', 'batting_hand']].drop_duplicates().sort_values(by='playerID').iterrows():
    print("    events_df.loc[events_df['batterID'] == '{0:}', 'batter_hand'] = '{1:}'".format(thisrow['playerID'], thisrow['batting_hand']))

for ix, thisbatterID in events_df[
    events_df['batter_hand'].isnull()
]['batterID'].sort_values().drop_duplicates().iteritems():
    print(thisbatterID)

print('\n')
print('No pitcher hand found for {0:,.0f} pitchers...'.format(len(events_df[events_df['pitcher_hand'].isnull()]['pitcherID'].drop_duplicates())))
print('\n')
# print(events_df.groupby('pitcher_hand').size())

missing_hand_pitchers_list = events_df[(events_df['pitcher_hand'].isnull())]['pitcherID'].drop_duplicates().sort_values().tolist()
#missing_hand_pitchers_list

for ix, thisrow in rosters_df[
    (rosters_df['playerID'].isin(missing_hand_pitchers_list)) 
    ][['playerID', 'throwing_hand']].drop_duplicates().sort_values(by='playerID').iterrows():
    print("    events_df.loc[events_df['pitcherID'] == '{0:}', 'pitcher_hand'] = '{1:}'".format(thisrow['playerID'], thisrow['throwing_hand']))

for ix, thispitcherID in events_df[
    events_df['pitcher_hand'].isnull()
]['pitcherID'].sort_values().drop_duplicates().iteritems():
    print(thispitcherID)

print('Done!')

No batter hand found for 0 batters...


No pitcher hand found for 0 pitchers...


Done!


In [5]:
s = time.time()
print('saving data...')
if (len(just_this_team) > 0):
    game_info_df.to_csv('game_info_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
    er_df.to_csv('er_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
    events_df.to_csv('events_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
#     events_df[events_df.index.isin(badj_at_events)].to_csv('badj_events_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
#     events_df[events_df.index.isin(padj_at_events)].to_csv('padj_events_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
#     events_df[events_df.index.isin(ladj_at_events)].to_csv('ladj_events_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
else:
    game_info_df.to_csv('game_info_{0:}_{1:}{2:}.csv'.format(firstyear, finalyear, league.lower()), encoding='utf-8')
    er_df.to_csv('er_{0:}_{1:}{2:}.csv'.format(firstyear, finalyear, league.lower()), encoding='utf-8')
    events_df.to_csv('events_{0:}_{1:}{2:}.csv'.format(firstyear, finalyear, league.lower()), encoding='utf-8')
#     events_df[events_df.index.isin(badj_at_events)].to_csv('badj_events_{0:}_{1:}{2:}.csv'.format(firstyear, finalyear, league), encoding='utf-8')
#     events_df[events_df.index.isin(padj_at_events)].to_csv('padj_events_{0:}_{1:}{2:}.csv'.format(firstyear, finalyear, league), encoding='utf-8')
#     events_df[events_df.index.isin(ladj_at_events)].to_csv('ladj_events_{0:}_{1:}{2:}.csv'.format(firstyear, finalyear, league), encoding='utf-8')

e = time.time()
g = g + (e-s)
print('Done in {0:,.0f} minutes {1:,.0f} seconds!'.format(np.floor(g/60), g%60))


saving data...
Done in 17 minutes 6 seconds!


In [6]:
# def fix_special_case_hands(events_df):
#     events_df.loc[events_df['batterID'] == 'andeb006', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'avila001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'barrf001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'bellc002', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'bichb001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'bogax001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'boted002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'braur002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'brinl001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'brows003', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'brucj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'buted001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'calhw001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'camaj001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'cavej001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'chavm001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'choij001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'corrc001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'crawb001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'crawj002', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'davij006', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'deshd002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'dickc002', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'dietd001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'difow001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'encae001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'fordm002', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'fralj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'freid001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'gallc001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'galvf001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'gonze004', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'gossp001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'grany001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'gricr001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'gurry001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'heywj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'hillt002', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'hiurk001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'holab001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'hoskr001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'jonej006', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'kiebc001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'lastt001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'lopet001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'marij002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'martj009', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'marts002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'mccua001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'mckib001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'mcmar001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'meada001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'moliy001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'mullc002', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'murps001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'myerw001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'solan001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'starb001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'susaa001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'tatif002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'verda001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'villj001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'voged001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'walkn001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'walsj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'wickr001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'yelic001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'zunim001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'basal001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'cronj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'garcl006', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'robel002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'ruizk001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'sancj003', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'woodj003', 'batter_hand'] = 'R'

#     # pitchers
#     events_df.loc[events_df['pitcherID'] == 'abrea001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'adamj002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'alcaj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'almoy001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'altad001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'alvaj004', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'anden001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'andes002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'andrm001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'antot001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'baezp001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'banda001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'barac001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'bardd001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'barnm001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'basht001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'bassa001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'bedrc001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'beekj001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'biagj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'bielb001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'blacr001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'brada001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'britz001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'brogc001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'buchr001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'bumma001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'burdn001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'buted001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'buttt001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'carrc004', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'castd002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'castm002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'cedeb001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'cessl001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'chafa001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'chapa001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'chavj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'cishs001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'cisnj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'claua001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'colea002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'coloa001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'coons001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'cordj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'could001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'crics001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'crocg001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'curtj003', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'davia003', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'davim005', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'daviw001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'daytg001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'detwr001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'diaze006', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'diazj006', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'dolir001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'dools001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'drako001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'dufft001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'duggr001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'espip001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'fairp001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'famij001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'farmb001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'feyej001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'flord002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'fontw001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'fostm001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'fry-j001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'fry-p001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'gallg001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'garcb001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'garcj006', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'garcl005', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'garcl007', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'garcr007', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gausk001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gearc001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gibai001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gilms001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'givem001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gomba001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'gonst001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gonzc002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gouda001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gracm003', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'gratb002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'greec003', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'grees005', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'grimj002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gselr001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'guerj002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'guerj003', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hadej001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'hahnj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hallm002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'handb001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'harpr001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'harvh001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'harvm001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hatct001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hellb001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'helsr001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hembh001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hendl001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hergj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hernj004', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hessd001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hillt002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'hiray001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hoffj003', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'holab001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'holdj002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hollg001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'housa001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hoytj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hudsd001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hughj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'igler001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'jansk001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'jeffj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'jimej003', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'johnp001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'jonen001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'kamir001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'karij001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'kellc002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'kellj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'kickm001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'kimbc001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'kinlt001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'kintb001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'kolaa001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'krate001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'lakit001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'leond003', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'lewia001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'lin-t001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'lindj004', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'loaij001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'lockw001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'lopet001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'lopey001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'lorem002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'loupa001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'lugos001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'luzaj001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'mahlt001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mantj002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'margn001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'martb002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'martc007', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'matop002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'matzs001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'matzt001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'may-t001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mayem001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mazzc002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mccak002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mcclr001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mcfat001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'mcgej001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'mcgok002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mcraa001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'melam001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'menec001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'mengd001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'milew001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'milla002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'millt003', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'milnh001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'minta001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'montr004', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'morab001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'morga001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'mujij001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'nelsn001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'nerih001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'osicj001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'pagae001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'patil001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'penaf002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'peraf001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'peraw002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'perda001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'pereo002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'pheld001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'phile001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'poncc002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'popps001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'presr001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'quanc001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'raleb001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'ramie004', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'ramiy002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'reedc002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'reyea004', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'riosy001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'roblh001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'roget001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'roget002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'romaj004', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'romej002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'romos001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'rosar002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'roset001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'sanac001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'sancr003', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'santd003', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'schrj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'scott003', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'scrua001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'selms001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'sherr001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'slega001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'smitd009', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'smitj004', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'smitr005', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'sneec001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'sorij001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'sotog003', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'strac001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'strih001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'suerw002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'sulsc001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'swane001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'taylb004', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'thiec001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'thomr007', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'thorl002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'tomlj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'treib001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'trivl001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'turln001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'unded001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'uriaj001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'valdc003', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'vincn001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'volqe001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'waddb001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'waldm002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'walkn001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'webbl001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'weber001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'wendj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'wickr001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'wilsj004', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'wislm001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'wittn001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'wojca001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'wooda002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'workb001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'yacaj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'yamas001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'yarde001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'youna002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'zeuct001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'zimmk001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'zubet001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'bednd001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'brayb002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'brubj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'burdz001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'burrb002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'casta005', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'crisn001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'elles001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'grotz001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hernc006', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'kingj002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'nelsk001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'weigp001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'woodj003', 'pitcher_hand'] = 'R'
    
#     # 2019 ANA
#     events_df.loc[events_df['batterID'] == 'astuw001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'cahit001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'castn001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'dozih001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'engea001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'freem002', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'gorda001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'hernt002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'kierk001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'luplj001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'moord002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'simma001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'beckg001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'hernm003', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'skolm001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'bassc001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'cahit001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'cimba001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'delpm001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'eliar001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'harrw002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'juraa001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'magim001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'magim001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'peacb001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'verlj001', 'pitcher_hand'] = 'R'    
#     events_df.loc[events_df['pitcherID'] == 'fierm001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'germd001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'sampa001', 'pitcher_hand'] = 'R'
    

#     # 2019 ARI
#     events_df.loc[events_df['batterID'] == 'arrij001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'austt001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'bertj001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'bleir001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'bleir001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'bradj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'cainl001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'carpm002', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'castm002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'choos001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'desca001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'donaj001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'flord002', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'forsl001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'fowld001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'fraza001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'frazt001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'frazt001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'freek001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'gameb001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'gracm003', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'greiz001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'herne001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'hosme001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'kinlt001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'meanj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'milla002', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'muncm001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'parrg001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'realj001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'reynb001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'senza001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'smitd007', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'smitj006', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'smitj006', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'suzuk001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'thame001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'uriaj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'vanmj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'vanmj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'waldm002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'brada001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'bettc001', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'blana002', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'davit004', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'duplj001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'iannc001', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'leybd001', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'martn003', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'mccab002', 'batter_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'arrij001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'austt001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'bergt001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'bergt001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'bleir001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'bleir001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'bradj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'cainl001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'carpm002', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'choos001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'desca001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'donaj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'frazt001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'frazt001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'freek001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'gameb001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'greiz001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'greiz001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'kuhnj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'meanj001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'parrg001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'reynb001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'ryank002', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'santh001', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'senza001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'smitj006', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'smitj006', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'stocr001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'thame001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'walkt004', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'walkt004', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'bettc001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'camaj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'duplj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'iannc001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'jackj001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'leybd001', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'martn003', 'pitcher_hand'] = 'L'


#.    1915




#     return events_df
# print('ok')